In [2]:
# Load the data and libraries
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

def laplace_mech(v, sensitivity, epsilon):
    return v + np.random.laplace(loc=0, scale=sensitivity / epsilon)

def gaussian_mech(v, sensitivity, epsilon, delta):
    return v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)

def gaussian_mech_vec(vec, sensitivity, epsilon, delta):
    return [v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)
            for v in vec]

In [3]:
# Load the data and libraries
import pandas as pd
import numpy as np

bear = pd.read_csv('https://raw.githubusercontent.com/jbennett979/Data_Privacy_FP/refs/heads/main/north_america_bear_killings.csv')

In [4]:
#removing space from the column name age\n",
bear.columns = [c.lstrip() for c in bear]
# capitalizing columns name for age and gende
bear= bear.rename(columns={'age': 'Age'})
bear= bear.rename(columns={'gender': 'Gender'})

In [6]:
print(bear.columns)
    # converting the age column who was a string to integers, while also adding error handling
bear['Age'] = pd.to_numeric(bear['Age'], errors = 'coerce')

Index(['Name', 'Age', 'Gender', 'Date', 'Month', 'Year', 'Type', 'Location',
       'Description', 'Type of bear', 'Hunter', 'Grizzly', 'Hikers',
       'Only one killed'],
      dtype='object')


In [7]:
 # starting with choosing a clip param using clip b param for age

def age_pick_b(epsilon):
   bs =  range(1,200,10)
   last_result = 0
   epsilon_i = epsilon / len(bs)

   for b in bs:
    #try b
    clipped_sum = bear['Age'].clip(upper=b).sum()
    result= (laplace_mech(clipped_sum, sensitivity=b, epsilon=epsilon_i))

    if result < last_result:
        return b
    else:
        last_result = result

    #raise Exception('No good clipping parameter found')

age_pick_b(1.0)

41

In [8]:
def year_pick_b(epsilon):
   bs =  range(1,200,10)
   last_result = 0
   epsilon_i = epsilon / len(bs)

   for b in bs:
    #try b
    clipped_sum = bear['Age'].clip(upper=b).sum()
    result= (laplace_mech(clipped_sum, sensitivity=b, epsilon=epsilon_i))

    if result < last_result:
        return b
    else:
        last_result = result

    #raise Exception('No good clipping parameter found')

pick_b(1.0)

41

In [13]:
# differential private year where most bear killing happened
def year_sum (epsilon):
   b = 100
   clipped_sum = bear['Year'].clip(upper=b).sum()

   return (laplace_mech(clipped_sum, sensitivity=b, epsilon=epsilon))

year_sum(1.0)

np.float64(16321.168777656632)

In [14]:
def dp_year_average(epsilon):

    noisy_sum = year_sum(epsilon/2)
    noisy_count = laplace_mech(len(bear), sensitivity=1, epsilon=epsilon/2)
    mean = noisy_sum/noisy_count
    return mean

dp_year_average(1.0)

np.float64(102.45743193042112)